---
title: Overview of the event
---

In [1]:
#| hide
%load_ext autoreload
%autoreload 2

In [11]:
from sunpy.time import TimeRange
from space_analysis.missions.psp.plasma import load_data as load_data_plasma
from space_analysis.plasma.formulary import df_beta, df_Alfven_speed
from space_analysis.utils.basic import resample
from beforerr.polars import pl_norm

from datetime import timedelta

In [3]:
start = '2019-04-06T12:00'
end = '2019-04-07T12:00'

earth_start = '2019-04-09'
earth_end = '2019-04-14'

In [4]:
def validate(timerange):
    if isinstance(timerange, TimeRange):
        return [timerange.start.to_string(), timerange.end.to_string()]

In [39]:
psp_timerange = TimeRange(start, end)

psp_plasma = load_data_plasma(trange = validate(psp_timerange), var_names=["DENS", "VEL_RTN_SUN", "TEMP", "MAGF_SC", "SUN_DIST"])

01-Feb-24 13:16:34: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spi/l3/spi_sf00_l3_mom/2019/


Using LEVEL=L3


01-Feb-24 13:16:34: File is current: /Users/zijin/data/psp/sweap/spi/l3/spi_sf00_l3_mom/2019/psp_swp_spi_sf00_l3_mom_20190406_v04.cdf
01-Feb-24 13:16:34: File is current: /Users/zijin/data/psp/sweap/spi/l3/spi_sf00_l3_mom/2019/psp_swp_spi_sf00_l3_mom_20190407_v04.cdf


In [40]:
vec_cols=[f"VEL_RTN_SUN_{i}" for i in range(3)]
magf_cols=[f"MAGF_SC_{i}" for i in range(3)]

In [59]:
import astropy.units as u
from plasmapy.formulary import beta, Alfven_speed
from plasmapy.particles import ParticleLike
import polars as pl

from typing import Optional


# | export
def df_beta(
    df: pl.DataFrame,
    T: str = "T",  # temperature of the plasma
    n: str = "n",  # particle density of the plasma
    B: str = "B",  # magnetic field in the plasma,
    col_name="beta",
    T_unit: u.Unit = u.eV,
    n_unit: u.Unit = u.cm**-3,
    B_unit: u.Unit = u.nT,
) -> pl.DataFrame:
    _T = df[T].to_numpy() * T_unit
    _n = df[n].to_numpy() * n_unit
    _B = df[B].to_numpy() * B_unit

    _beta = beta(T=_T, n=_n, B=_B)

    return df.with_columns(pl.Series(_beta).alias(col_name))


# | export
def df_Alfven_speed(
    df: pl.DataFrame,
    B: str = "B",  # magnetic field in the plasma
    n: str = "n",  # particle density of the plasma
    ion: ParticleLike = "p+",
    col_name="Alfven_speed",
    B_unit: u.Unit = u.nT,
    n_unit: u.Unit = u.cm**-3,
    speed_unit: u.Unit = u.km / u.s,
):
    _B = df[B].to_numpy() * B_unit
    _n = df[n].to_numpy() * n_unit

    _Alfven_speed = Alfven_speed(B=_B, density=_n, ion=ion).to(speed_unit)

    return df.with_columns(pl.Series(_Alfven_speed).alias(col_name))

## Radial evolution of the coronalhole high-speed streams

The radial dependence of the proton number density, np is

$$n_p = (2.4 ± 0.1)(R/R_0)^{−(1.96±0.07)} cm^{−3}$$

$$B = (5.7 ± 0.2)(R/R_0)^{−(1.59±0.06)} nT$$

The faster decrease of the magnetic than kinetic pressure is reﬂected in the radial proton plasma beta variation

$$β_p = P_k/P_B = (0.55 ± 0.04)(R/R_0)^{(0.4±0.1)}.$$

The behaviour of the parallel proton plasma beta is similar 

$$β_{‖} = (0.37 ± 0.03)(R/R_0)(0.8±0.1)$$

In [85]:
# The radial dependence of the proton number density, np is

# $$n_p = (2.4 ± 0.1)(R/R_0)^{−(1.96±0.07)} cm^{−3}$$

# $$B = (5.7 ± 0.2)(R/R_0)^{−(1.59±0.06)} nT$$

# The faster decrease of the magnetic than kinetic pressure is reﬂected in the radial proton plasma beta variation

# $$β_p = P_k/P_B = (0.55 ± 0.04)(R/R_0)^{(0.4±0.1)}.$$

# The behaviour of the parallel proton plasma beta is similar 

# $$β_{‖} = (0.37 ± 0.03)(R/R_0)(0.8±0.1)$$

def plasma_r_evolution(
    df: pl.DataFrame,
    alpha_beta_r = 0.4,
    alpha_beta_parallel_r = 0.9,
    alpha_n = -2,
    alpha_B = -1.63,
    alpha_plasma_speed = 0,
):
    return df.with_columns(
        plasma_speed_1AU = pl.col('plasma_speed') * (1 / pl.col("distance2sun"))**alpha_plasma_speed,
        n_1AU = pl.col('n') * (1 / pl.col("distance2sun"))**alpha_n,
        B_1AU = pl.col('B') * (1 / pl.col("distance2sun"))**alpha_B,
        beta_1AU = pl.col("beta") * (1 / pl.col("distance2sun"))**alpha_beta_r,
        beta_parallel = pl.col("beta") * (1 / pl.col("distance2sun"))**alpha_beta_parallel_r,
    )

In [86]:
every = timedelta(minutes=4)

km2au = u.km.to(u.AU)

psp_plasma_r = (
    psp_plasma.pipe(resample, every)
    .with_columns(
        B=pl_norm(magf_cols),
        plasma_speed = pl_norm(vec_cols),
        distance2sun = pl.col('SUN_DIST') * km2au,
    )
    .rename(
        {
            "DENS": "n",
            "TEMP": "T",
        }
    )
    .collect()
    .pipe(df_beta)
    .pipe(plasma_r_evolution)
    .pipe(df_Alfven_speed)
    .pipe(df_Alfven_speed, B = "B_1AU", n = "n_1AU", col_name = "Alfven_speed_1AU")
    .with_columns(
        plasma_speed_over_Alfven_speed = pl.col("plasma_speed") / pl.col("Alfven_speed"),
        plasma_speed_over_Alfven_speed_1AU = pl.col("plasma_speed_1AU") / pl.col("Alfven_speed_1AU"),
    )
)

In [89]:
plot = psp_plasma_r.plot(x="time", y=["plasma_speed", "beta", "Alfven_speed", "plasma_speed_over_Alfven_speed"], subplots=True, shared_axes=False).cols(1)

plot_1AU = psp_plasma_r.plot(x="time", y=["plasma_speed_1AU", "beta_1AU", "Alfven_speed_1AU", "plasma_speed_over_Alfven_speed_1AU"], subplots=True, shared_axes=False).cols(1)

plot + plot_1AU

:Layout
   .NdLayout.I  :NdLayout   [Variable]
      :Curve   [time]   (value)
   .NdLayout.II :NdLayout   [Variable]
      :Curve   [time]   (value)

In [88]:
(psp_plasma_r.plot.scatter(x="beta", y="plasma_speed_over_Alfven_speed", logx=True, logy=True) + psp_plasma_r.plot.scatter(x="beta_1AU", y="plasma_speed_over_Alfven_speed_1AU", logx=True, logy=True)).cols(1)

:Layout
   .Scatter.I  :Scatter   [beta]   (plasma_speed_over_Alfven_speed)
   .Scatter.II :Scatter   [beta_1AU]   (plasma_speed_over_Alfven_speed_1AU)